In [1]:
# from pylabel import importer
import glob
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import yaml
import os
import glob
import re
import folium

from shapely.geometry import box, Polygon, MultiPolygon, Point

from shapely.validation import make_valid
import glob
import sys
from matplotlib import patches
import geopandas as gpd
import rasterio
import rasterio.transform
from rasterio.plot import show
%config InlineBackend.figure_format='retina'

# Look the shape file

In [3]:
STATE = 'Ondo'
annot =  glob.glob(f"/lustre/davis/FishPonds_project/share/data/{STATE}/*geocoords.shp")
print(annot)

['/lustre/davis/FishPonds_project/share/data/Ondo/Ondo_all_geocoords.shp']


If print(annot) return [], that means the Job failed.

# Load the shape file

In [4]:
dest = gpd.read_file(annot[0])

In [ ]:
dest.shape

# Extract the number of the grids that contribute to the predictions

In [5]:
where_shape = glob.glob(f"/lustre/davis/FishPonds_project/share/data/{STATE}/*/geocoords/*.shp")
grids = [int(x.split('/')[-1].split('_')[1]) for x in where_shape]

# Plot the predictions, and also show the grid

In [7]:
centroids = dest.centroid.to_crs("epsg:4326")
zoom_centroid_idx = 1
m = folium.Map(
    location=[centroids.iloc[zoom_centroid_idx].y, centroids.iloc[zoom_centroid_idx].x],
    zoom_start=25,
    tiles=None  # Start with no tiles
)


# Add Google Satellite tileset
tile = folium.TileLayer(
    tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
    attr="Google",
    name="Google Satellite",
    overlay=False
)
# tile.add_to(m)
tile.add_to(m)
dest.explore(column="class",style_kwds=dict(color="gold",weight=2, opacity=1, fillOpacity=.2), m=m)

#plot in gray all the grids that filled the state
grid = gpd.read_file(f"data/{STATE}/grid_{STATE}.shp")
grid.explore(style_kwds=dict(color="gray",weight=2, opacity=1, fillColor='#00000000', fillOpacity=.0), m=m)
# plot in red the grids that have predictions
for g in grids:
    grid.iloc[g:g+1, :].explore(style_kwds=dict(color="crimson",weight=2, opacity=1, fillColor='#00000000', fillOpacity=.0), m=m)

m